# Data Load
[사용 데이터 : Kaggle eCommerce Event Hisotry in Cosmetic Shop](https://www.kaggle.com/mkechinov/ecommerce-events-history-in-cosmetics-shop).

# RFM Analysis
#### RFM(Recency, Frequency, Monetary) 분석은 새로운 제안에 응답할 가능성이 가장 높은 기존 고객을 식별하는 데 사용되는 방법임. 이 방법은 주로 다이렉트 마케팅에서 사용됨.
- **Recency** : 가장 최근 구매 날짜 또는 가장 최근 구매 날짜 이후의 시간 간격. 이 정보는 최근성 점수를 계산하는 데 사용됨.
- **Frequency** : 총 구매 수. 이 정보는 빈도 점수를 계산하는 데 사용됨.
- **Monetary** : 모든 구매의 구매총액 집계값. 이 정보는 구매총액 점수를 계산하는 데 사용됨.

In [2]:
import pandas as pd
import numpy as np
import os

In [3]:
import warnings
warnings.filterwarnings(action='ignore')

In [4]:
data0 = pd.read_csv('2019-Oct-Copy1.csv')
data1 = pd.read_csv('2019-Nov-Copy1.csv')
data2 = pd.read_csv('2019-Dec-Copy1.csv')
data3 = pd.read_csv('2020-Jan-Copy1.csv')
data4 = pd.read_csv('2020-Feb-Copy1.csv')

In [5]:
temp = pd.concat([data1,data2,data3,data4])

del [[data1, data2, data3, data4]]

# (1) Get Purchase Data

In [6]:
purchase = temp.loc[temp.event_type == 'purchase']

In [7]:
purchase.head(10)

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
30,2019-11-01 00:01:57 UTC,purchase,5839412,1487580006551913373,NaN,lovely,3.16,460304619,9f777569-bdf3-47e5-a3d4-dfc26beb29cb
32,2019-11-01 00:01:57 UTC,purchase,5823969,1487580005268456287,NaN,uno,17.46,460304619,9f777569-bdf3-47e5-a3d4-dfc26beb29cb
33,2019-11-01 00:01:57 UTC,purchase,5810480,1487580011283087468,NaN,NaN,22.54,460304619,9f777569-bdf3-47e5-a3d4-dfc26beb29cb
61,2019-11-01 00:04:33 UTC,purchase,24380,1487580012994363565,NaN,depilflax,5.24,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f
62,2019-11-01 00:04:33 UTC,purchase,26765,1487580013522845895,NaN,ardell,7.16,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f
63,2019-11-01 00:04:33 UTC,purchase,5573498,1487580012969197740,NaN,candy,4.29,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f
64,2019-11-01 00:04:33 UTC,purchase,5586154,1958278551207674674,NaN,inm,3.17,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f
65,2019-11-01 00:04:33 UTC,purchase,5739918,1487580012994363565,NaN,italwax,8.73,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f
66,2019-11-01 00:04:33 UTC,purchase,5766980,1487580013053083824,stationery.cartrige,italwax,1.98,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f
67,2019-11-01 00:04:33 UTC,purchase,5767494,1487580013053083824,stationery.cartrige,italwax,2.14,564451209,861ab2f1-b2e5-886f-a93b-5b067eff081f


In [8]:
%%time
purchase['event_time']=pd.to_datetime(purchase['event_time']).dt.tz_convert(None)

CPU times: user 13 s, sys: 11.6 ms, total: 13 s
Wall time: 13 s


In [9]:
purchase_data=purchase.groupby(by='user_session').agg(Date_order=('event_time',lambda x: x.max()),
                                                  user_id=('user_id',lambda x: x.unique()),
                                          Quantity=('event_type','count'),
                                         money_spent=('price','sum')).reset_index(drop=True)
purchase_data

,Date_order,user_id,Quantity,money_spent
0,2020-02-15 09:11:59,544281930,4,6.83
1,2019-12-14 09:01:26,507234407,5,3.55
2,2019-12-29 06:23:22,594179781,1,20.00
3,2019-11-07 12:31:32,549507462,252,766.28
4,2019-12-13 21:55:34,502964284,8,52.07
...,...,...,...,...
126572,2020-01-31 05:20:41,568871996,13,40.91
126573,2019-12-24 07:20:04,505383393,9,31.22
126574,2020-02-21 20:02:10,618510291,5,9.60
126575,2020-02-12 08:28:55,231066621,2,9.70


# (2) Reference Date 기준 재정렬

In [10]:
purchase_data['Date_order'].max()

Timestamp('2020-02-29 23:54:44')

In [11]:
import datetime as dt
standard_date = dt.datetime(2020,3,1)
purchase_data = pd.DataFrame(purchase_data)
purchase_data['last_purchase'] = standard_date - purchase_data['Date_order']
purchase_data['last_purchase'].astype('timedelta64[D]')
purchase_data['last_purchase']=purchase_data['last_purchase'] / np.timedelta64(1, 'D')
purchase_data.head()

,Date_order,user_id,Quantity,money_spent,last_purchase
0,2020-02-15 09:11:59,544281930,4,6.83,14.616678
1,2019-12-14 09:01:26,507234407,5,3.55,77.624005
2,2019-12-29 06:23:22,594179781,1,20.00,62.733773
3,2019-11-07 12:31:32,549507462,252,766.28,114.478102
4,2019-12-13 21:55:34,502964284,8,52.07,78.086412


# (3) RFM 구간화
- R(Recency) : 기준일을 기준으로 마지막 구매일까지의 날
- F(Frequency) : 단위 기간 동안 상품 구매한 빈도
- M(Monetary) : 단위 기간 동안 구매 상품 가격 총량

In [12]:
#Calculate Recency, Frequency, and Monetary of the data
RFM= purchase_data.groupby('user_id').agg(Recency=('last_purchase',lambda x: x.min()),
                                 Frequency=('Quantity',lambda x: x.sum()),
                                 Monetary=('money_spent',lambda x: x.sum()))
RFM.head()

,Recency,Frequency,Monetary
user_id,,,
9794320,96.786655,4,12.68
10079204,115.553125,2,25.81
10280338,10.360058,86,177.83
12055855,71.496192,4,16.54
12936739,43.672697,2,29.89


In [13]:
RFM['Recency'].describe()

count    92684.000000
mean        57.369404
std         35.642288
min          0.003657
25%         25.711991
50%         53.482569
75%         91.071956
max        120.977928
Name: Recency, dtype: float64

In [14]:
RFM['Frequency'].describe()

count    92684.000000
mean        11.235844
std         16.999654
min          1.000000
25%          3.000000
50%          6.000000
75%         13.000000
max        563.000000
Name: Frequency, dtype: float64

# (4) Scoring RFM With Quantile

In [15]:
RFM.quantile(q=[0.25,0.5,0.75])

,Recency,Frequency,Monetary
0.25,25.711991,3.0,16.19
0.50,53.482569,6.0,32.97
0.75,91.071956,13.0,60.48


In [16]:
quartiles=RFM.quantile(q=[0.25,0.5,0.75]).to_dict()
quartiles

{'Recency': {0.25: 25.71199074074074,
  0.5: 53.482569444444444,
  0.75: 91.07195601851852},
 'Frequency': {0.25: 3.0, 0.5: 6.0, 0.75: 13.0},
 'Monetary': {0.25: 16.19, 0.5: 32.97, 0.75: 60.48}}

In [17]:
def R_Score(x,p,d):
    if x<= d[p][0.25]: return 4
    elif x <= d[p][0.50]: return 3
    elif x <= d[p][0.75]: return 2
    else : return 1

def FM_Score(x,p,d):
    if x<= d[p][0.25]: return 1
    elif x <= d[p][0.50]: return 2
    elif x <= d[p][0.75]: return 3
    else : return 4

In [18]:
# create RFM segmentation column
RFM['R_Quartile'] = RFM['Recency'].apply(R_Score, args=('Recency',quartiles,))
RFM['F_Quartile'] = RFM['Frequency'].apply(FM_Score, args=('Frequency',quartiles,))
RFM['M_Quartile'] = RFM['Monetary'].apply(FM_Score, args=('Monetary',quartiles,))
RFM['RFM_score'] = RFM.R_Quartile.map(int)+ RFM.F_Quartile.map(int)+RFM.M_Quartile.map(int)
RFM.head()

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFM_score
user_id,,,,,,,
9794320,96.786655,4,12.68,1,2,1,4
10079204,115.553125,2,25.81,1,1,2,4
10280338,10.360058,86,177.83,4,4,4,12
12055855,71.496192,4,16.54,2,2,2,6
12936739,43.672697,2,29.89,3,1,2,6


In [19]:
RFM['RFM_score'].describe()

count    92684.000000
mean         7.401148
std          2.475192
min          3.000000
25%          5.000000
50%          7.000000
75%          9.000000
max         12.000000
Name: RFM_score, dtype: float64

In [20]:
RFM[RFM['RFM_score']==12]

,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFM_score
user_id,,,,,,,
10280338,10.360058,86,177.83,4,4,4,12
47076222,4.040671,29,84.10,4,4,4,12
50764735,21.417350,41,182.57,4,4,4,12
54710988,14.058067,26,120.18,4,4,4,12
56612519,23.510301,78,225.82,4,4,4,12
...,...,...,...,...,...,...,...
621849967,0.520428,60,670.72,4,4,4,12
621911648,0.215729,40,130.45,4,4,4,12
621915467,0.381262,27,85.64,4,4,4,12


In [21]:
RFM = RFM.reset_index().rename(columns={"index":"user_id"})
RFM

,user_id,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFM_score
0,9794320,96.786655,4,12.68,1,2,1,4
1,10079204,115.553125,2,25.81,1,1,2,4
2,10280338,10.360058,86,177.83,4,4,4,12
3,12055855,71.496192,4,16.54,2,2,2,6
4,12936739,43.672697,2,29.89,3,1,2,6
...,...,...,...,...,...,...,...,...
92679,622065819,0.062975,4,20.48,4,2,2,8
92680,622066161,0.087245,4,12.95,4,2,1,7
92681,622067983,0.059988,10,66.48,4,3,4,11
92682,622069477,0.079514,1,0.95,4,1,1,6


# (5) Get User Data Via RFM Score

In [22]:
user_8 = RFM[RFM['RFM_score']==8]
user_8

,user_id,Recency,Frequency,Monetary,R_Quartile,F_Quartile,M_Quartile,RFM_score
7,19762782,111.193044,8,80.29,1,3,4,8
8,20554973,45.324664,7,21.81,3,3,2,8
9,23633626,32.618634,8,22.13,3,3,2,8
10,25300038,3.477083,5,28.70,4,2,2,8
14,27872944,15.652025,5,18.71,4,2,2,8
...,...,...,...,...,...,...,...,...
92652,622016893,0.211157,1,53.17,4,1,3,8
92658,622020297,0.192326,2,35.78,4,1,3,8
92659,622020692,0.197581,1,42.86,4,1,3,8
92677,622058876,0.095278,3,45.24,4,1,3,8


In [23]:
user8_list = user_8['user_id'].tolist()

mid_user = temp[temp['user_id'].isin(user8_list)]

In [24]:
mid_user.head()

,event_time,event_type,product_id,category_id,category_code,brand,price,user_id,user_session
308,2019-11-01 00:23:52 UTC,view,5628025,1605161575889502297,NaN,NaN,2.70,462885785,5dc3598b-614c-45e5-8a52-2a6bb306a9b9
315,2019-11-01 00:24:28 UTC,view,5885418,1487580005092295511,NaN,grattol,6.27,462885785,5dc3598b-614c-45e5-8a52-2a6bb306a9b9
469,2019-11-01 00:36:02 UTC,cart,5597836,1487580005713052531,NaN,ingarden,2.60,469589164,292f045b-fde9-41a8-8b4f-daaee93528f1
471,2019-11-01 00:36:11 UTC,view,5760173,1487580009387261981,NaN,ingarden,4.29,469589164,292f045b-fde9-41a8-8b4f-daaee93528f1
517,2019-11-01 00:39:33 UTC,view,5877451,1487580006300255120,NaN,jessnail,44.29,451374803,edf7c777-4e98-49c9-89d7-bb5afbc451da
